In [86]:
import urllib.parse
import requests
from owlready2 import *
from datetime import datetime, timedelta, date
import csv
import random
from bs4 import BeautifulSoup

theaters = []
titles = []
number_of_movies_per_theater = []

def get_movies():
    with open('./movieList.csv') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            titles.append(row[0])

def convert_theater():
    # send the API request
    response = requests.get('https://data.culture.gouv.fr/api/records/1.0/search/?dataset=etablissements-cinematographiques&q=&lang=en&rows=10&facet=region_administrative&facet=genre&facet=multiplexe&facet=zone_de_la_commune')

    data = response.json()
    
    if data.get("Response") == "False":
        print(data.get("Error"))
    else:
        onto = get_ontology("C:/Users/alexa/Downloads/movie-version1.owl").load()

        for records in data['records']:
            attributes = records['fields']

            if("'" in attributes['nom']):
                continue

            number_of_movies_per_theater.append(int(attributes["nombre_de_films_programmes"]))
            theaters.append(attributes['nom'].title().replace(" ", "_"))

            if([attributes["multiplexe"]] == ['OUI']):
                theater = onto.Multiplex(attributes['nom'].title().replace(" ", "_"))
                theater.entriesNumber = [attributes["tranche_d_entrees"]]
                theater.city = [attributes["commune"]]
                theater.theaterName = [attributes["nom"].capitalize()]
                theater.owner = [attributes["proprietaire"].capitalize()]
                theater.sessions = [int(attributes["seances"])]
                theater.region = [attributes["region_administrative"].capitalize()]
                theater.address = [attributes["adresse"].capitalize()]
                theater.screens = [int(attributes["ecrans"])]
                theater.seats = [int(attributes["fauteuils"])]
                theater.numberOfMovies = [int(attributes["nombre_de_films_programmes"])]
                theater.numberOfMoviesPerWeek = [int(attributes["nombre_de_films_en_semaine_1"])]

            else:
                theater = onto.Theater(attributes['nom'].title().replace(" ", "_"))
                theater.entriesNumber = [attributes["tranche_d_entrees"]]
                theater.city = [attributes["commune"]]
                theater.theaterName = [attributes["nom"].capitalize()]
                theater.owner = [attributes["proprietaire"].capitalize()]
                theater.sessions = [int(attributes["seances"])]
                theater.region = [attributes["region_administrative"].capitalize()]
                theater.address = [attributes["adresse"].capitalize()]
                theater.screens = [int(attributes["ecrans"])]
                theater.seats = [int(attributes["fauteuils"])]
                theater.numberOfMovies = [int(attributes["nombre_de_films_programmes"])]
                theater.numberOfMoviesPerWeek = [int(attributes["nombre_de_films_en_semaine_1"])]
        
        onto.save("C:/Users/alexa/Downloads/movie-version1.owl", format="rdfxml")

def convert_movie(title):

    try:
        params = {'t': title, 'plot': 'short', 'r': 'json'}
        
        response = requests.get('http://www.omdbapi.com/?apikey=eda75ca', params=params)

        data = response.json()

        if data.get("Response") == "False":
            print(data.get("Error"))
        else:
            onto = get_ontology("C:/Users/alexa/Downloads/movie-version1.owl").load()
            
            movie = onto.Movie(title.replace(" ", "_"))

            movie.title = [data["Title"]]
            movie.year = [int(data["Year"])]
            movie.country = [data["Country"]]
            movie.genre = data["Genre"].split(", ")
            movie.language = data["Language"].split(", ")
            movie.released = [datetime.strptime(data["Released"], '%d %b %Y')]
            movie.runtime = [int(re.findall(r'\d+', data["Runtime"])[0])]
            movie.rated = [data["Rated"]]
            movie.plot = [data["Plot"]]
            movie.awards = [data["Awards"]]
            movie.poster = [data["Poster"]]
            movie.metascore = [int(data["Metascore"])]
            movie.imdbRating = [float(data["imdbRating"])]
            movie.imdbID = [data["imdbID"]]
            movie.endDate, movie.startDate = setDate()

            if(len(movie.BoxOffice) != 0):
                movie.BoxOffice = [int(data["BoxOffice"].replace("$", "").replace(",", ""))]

            for director in data["Director"].split(", "):
                d = onto.Director(director.replace(" ", "_"))
                d.birthdate, d.age = birthday(director.replace(" ", "_"))
                d.fullname = [director.replace("_", "")]
                movie.hasDirector.append(d)
                
            for writer in data["Writer"].split(", "):
                w = onto.Writer(writer.replace(" ", "_"))
                w.birthdate, w.age = birthday(writer.replace(" ", "_"))
                w.fullname = [writer.replace("_", "")]
                movie.hasWriter.append(w)

            for actor in data["Actors"].split(", ") :
                a = onto.Actor(actor.replace(" ", "_"))
                a.birthdate, a.age = birthday(writer.replace(" ", "_"))
                a.fullname = [actor.replace("_", "")]
                movie.hasActor.append(a)

        onto.save("C:/Users/alexa/Downloads/movie-version1.owl", format="rdfxml")
    
    except Exception as e:
        print(e)

def setDate():
    start_date = datetime(2022, 10, 1, 0, 0, 0)
    end_date = date.today()

    time_between_dates = end_date - start_date.date()
    days_between_dates = time_between_dates.days

    dateOne = random.randrange(days_between_dates)
    dateTwo = random.randrange(days_between_dates)

    randomStart = dateOne if dateTwo < dateOne else dateTwo
    randomEnd = dateOne if dateOne < dateTwo else dateTwo

    return [start_date + timedelta(days=randomStart)], [start_date + timedelta(days=randomEnd)]

def format(list):
    formatted = []

    for element in list:
        formatted.append(element.replace(" ", "_"))

    return formatted

def fill_theaters(instance):
    onto = get_ontology("C:/Users/alexa/Downloads/movie-version1.owl").load()

    theater = onto.Theater
    movie = onto.Movie

    numberOfMovies = list(range(0, theaters_dictionary[instance]))
    movieIndex = format(titles)

    for i in range(len(numberOfMovies)):
        randomMovie = random.sample(movieIndex, 1)[0]
        theater(instance).hasMovie.append(movie(randomMovie))
        movieIndex.remove(randomMovie)

    onto.save("C:/Users/alexa/Downloads/movie-version1.owl", format="rdfxml")

def birthday(FullName):
    now = datetime.now()

    try:
        result = requests.get("https://en.wikipedia.org/wiki/" + FullName)
        src = result.content
        soup = BeautifulSoup(src, 'lxml')
        bday = soup.find("span", {"class": "bday"})
        bday = (str(bday)[19:29])
        bbyear = int((str(bday)[0:4]))
        age = int(now.year) - bbyear
        bmonth = int((str(bday)[5:7]))
        bdday = int((str(bday)[8:10]))
        monthtemp = int(now.month) - bmonth
        daytemp = int(now.day) - bdday

        if daytemp == 0 and monthtemp == 0:
            age = age
        elif daytemp < 1 and monthtemp < 1:
            age = age - 1

        date_string = str(bdday) + "/" + str(bmonth) + "/" + str(bbyear)
        date_format = "%d/%m/%Y"

        birthdate = datetime.strptime(date_string, date_format)

        return [birthdate], [age]

    except:
        print("{FirstName} {LastName} has not been found on wikipedia.")

get_movies()

# loop over the movie titles and retrieve their information
for title in titles:
    convert_movie(title)

convert_theater()

theaters_dictionary = dict(zip(theaters, number_of_movies_per_theater))

for theater in theaters:
    fill_theaters(theater)

{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName} {LastName} has not been found on wikipedia.
cannot unpack non-iterable NoneType object
{FirstName

In [81]:
from owlready2 import *
onto = get_ontology("http://www.semanticweb.org/alexandrecogordan/ontologies/2023/1/untitled-ontology-26#").load()
Person=onto.Person
# Get all the individuals of the class Person
individuals = list(Person.instances())

# Print the individuals
for individual in individuals:
    print(individual.name)

AJ_Michalka
Dane_DeHaan
Jason_Isaacs
Mia_Goth
Catherine_Walker
Steve_Oram
Susan_Loughnane
Abbie_Cornish
Albert_Finney
Russell_Crowe
Omar_Elba
Sarita_Choudhury
Tom_Hanks
Haley_Bennett
Jessica_Biel
Patrick_Wilson
Charlize_Theron
Liam_Neeson
Seth_MacFarlane
Felicity_Jones
Lewis_MacDougall
Sigourney_Weaver
Emma_Bell
Rose_Williams
Sara_Vertongen
Bob_the_Cat
Luke_Treadaway
Ruta_Gedmintas
Aamir_Khan
Aaron_Burns
Aaron_Eckhart
Aaron_Paul
Aaron_Poole
Aaron_Taylor-Johnson
Bill_Nighy
Domhnall_Gleeson
Rachel_McAdams
Kate_Beckinsale
Sanjeev_Bhaskar
Simon_Pegg
Adam_Brody
Adam_Devine
Adam_Driver
Adam_Sandler
Adil_Hussain
Devon_Bostick
Louca_Tassone
Rachel_Blanchard
Adria_Arjona
Adrian_Grenier
Adrien_Brody
Adrienne_King
Aidan_Gillen
Akiva_Schaffer
Al_Pacino
Alain_Moussi
Alan_Alda
Alan_Rickman
Alan_Tudyk
Albert_Brooks
Alden_Ehrenreich
Aldis_Hodge
Alessandro_Nivola
Alex_Karpovsky
Alex_Pettyfer
Alexander_Skarsgård
Alexandra_Daddario
Alexi_Pappas
Alfie_Allen
Ali_Larter
Alia_Shawkat
Alice_Braga
Alice_Engler

In [65]:
from rdflib.plugins.sparql import prepareQuery

# Load the SPARQL query
with open("./data/query.rq") as f:
    query_text = f.read()
query = prepareQuery(query_text)

sync_reasoner()

# Execute the query
results = onto.sparql_query(str(query))

# Display the results
for result in results:
    print(result.name)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\Users\alexa\Anaconda3\lib\site-packages\owlready2\hermit;c:\Users\alexa\Anaconda3\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/alexa/AppData/Local/Temp/tmp3mk1npz9
* Owlready2 * HermiT took 6.573266267776489 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


TypeError: 'NoneType' object is not callable

In [83]:
from rdflib import Graph, URIRef, Literal,term,Namespace
from rdflib.namespace import RDF, RDFS, OWL

# Create a Graph, pare in Internet data
g = Graph().parse("C:/Users/alexa/Downloads/movie-version1.owl")

# Query the data in g using SPARQL
# This query returns the 'name' of all ``foaf:Person`` instances

g.bind("owl", OWL)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)

movie = Namespace('http://www.semanticweb.org/alexandrecogordan/ontologies/2023/1/untitled-ontology-26#')
g.bind('movie', movie)
q = """
    SELECT ?persons
    WHERE {
    ?persons rdf:type movie:Person . 
}
"""

# Execute the SPARQL query
results = g.query(query)


# Print the results
for person in results:
    print(person)




[]


In [52]:
Actor=onto.Actor

# Get all the individuals of the class Person
individuals = list(Actor.instances())

# Print the individuals
for individual in individuals:
    print(individual.name)

AJ_Michalka
Dane_DeHaan
Jason_Isaacs
Mia_Goth
Catherine_Walker
Steve_Oram
Susan_Loughnane
Abbie_Cornish
Albert_Finney
Russell_Crowe
Omar_Elba
Sarita_Choudhury
Tom_Hanks
Haley_Bennett
Jessica_Biel
Patrick_Wilson
Charlize_Theron
Liam_Neeson
Seth_MacFarlane
Felicity_Jones
Lewis_MacDougall
Sigourney_Weaver
Emma_Bell
Rose_Williams
Sara_Vertongen
Bob_the_Cat
Luke_Treadaway
Ruta_Gedmintas
Aamir_Khan
Aaron_Burns
Aaron_Eckhart
Aaron_Paul
Aaron_Poole
Aaron_Taylor-Johnson
Bill_Nighy
Domhnall_Gleeson
Rachel_McAdams
Kate_Beckinsale
Sanjeev_Bhaskar
Simon_Pegg
Adam_Brody
Adam_Devine
Adam_Driver
Adam_Sandler
Adil_Hussain
Devon_Bostick
Louca_Tassone
Rachel_Blanchard
Adria_Arjona
Adrian_Grenier
Adrien_Brody
Adrienne_King
Aidan_Gillen
Akiva_Schaffer
Al_Pacino
Alain_Moussi
Alan_Alda
Alan_Rickman
Alan_Tudyk
Albert_Brooks
Alden_Ehrenreich
Aldis_Hodge
Alessandro_Nivola
Alex_Karpovsky
Alex_Pettyfer
Alexander_Skarsgård
Alexandra_Daddario
Alexi_Pappas
Alfie_Allen
Ali_Larter
Alia_Shawkat
Alice_Braga
Alice_Engler

In [78]:
persons = []
for s, p, o in g:
    if p == term.URIRef('http://www.semanticweb.org/alexandrecogordan/ontologies/2023/1/untitled-ontology-26#fullname'):
        persons.append(str(o))
txt = "\n".join(persons)
print(txt)

Patricia Clarkson
Mara Leveritt
Scott Alexander
Paul Wernick
Stephen J. Rivele
Jacob Koskoff
Albert Brooks
Jackson Rathbone
Nikolaj Coster-Waldau
Ronan Bennett
Kit Harington
Will Smith
Rebel Wilson
Jemaine Clement
Ron Padgett
Erdman Penner
Sandra Bullock
Stephanie Fabrizi
Steven Rogers
Jonás Cuarón
Yoni Brenner
Larry Karaszewski
Gillies MacKinnon
Adrian Grenier
Adam Driver
Tony Mosher
Lenny Abrahamson
Justin Haythe
Lewis MacDougall
Simon Kelton
Maxim Gaudette
Luke Bracey
Joe Carnahan
Stieg Larsson
Simon Kinberg
Mark Strong
Mike Judge
David Guion
George MacKay
Chris Pappas
Michelle Rodriguez
Henry Joost
Noah Ringer
Nick Hornby
Ross Katz
Christopher Miller
Dave Johns
Laurent Lafitte
Domhnall Gleeson
Simon Carmody
Bo Goldman
Elizabeth Reaser
Jean-Christophe Castelli
Ellen Furman
Ray Stevenson
Joan Allen
Jason Moore
Todd Louiso
Alexander Dinelaris
Christopher Lee
John-Henry Butterworth
Sandra Hüller
Peter Laird
Meryl Streep
Colin Hanks
Dan Fogelman
Simon Verhoeven
Kathryn Bigelow
Jan Pinka